In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from nzdownscale.downscaler.validate_v2 import ValidateV2
from datetime import datetime, timedelta

In [3]:
model_name = 'hourly_v2'
var = 'temperature'

model_dir = f'/home/emily/deepsensor/deepweather-downscaling/experiments/'\
            f'models/{var}/{model_name}'
model_path = f'{model_dir}/{model_name}.pt'
train_metadata_path = f'{model_dir}/metadata_{model_name}.pkl'
data_processor_path = f'{model_dir}/data_processor.pkl'
task_loader_path = f'{model_dir}/task_loader.pkl'

In [8]:
validate = ValidateV2(model_path,
                      data_processor_path,
                      task_loader_path,
                      train_metadata_path)

Producing predictions at resolution: 0.01


In [9]:
time = [datetime(2010, 1, 1, i) for i in range(24)]

In [10]:
pred = validate.predict(time)

Loading ERA5:   0%|          | 0/6 [00:00<?, ?it/s]

Loading stations: 100%|██████████| 444/444 [00:40<00:00, 10.94it/s]


159 stations with data at ['2010-01-01T00:00:00.000000000' '2010-01-01T01:00:00.000000000'
 '2010-01-01T02:00:00.000000000' '2010-01-01T03:00:00.000000000'
 '2010-01-01T04:00:00.000000000' '2010-01-01T05:00:00.000000000'
 '2010-01-01T06:00:00.000000000' '2010-01-01T07:00:00.000000000'
 '2010-01-01T08:00:00.000000000' '2010-01-01T09:00:00.000000000'
 '2010-01-01T10:00:00.000000000' '2010-01-01T11:00:00.000000000'
 '2010-01-01T12:00:00.000000000' '2010-01-01T13:00:00.000000000'
 '2010-01-01T14:00:00.000000000' '2010-01-01T15:00:00.000000000'
 '2010-01-01T16:00:00.000000000' '2010-01-01T17:00:00.000000000'
 '2010-01-01T18:00:00.000000000' '2010-01-01T19:00:00.000000000'
 '2010-01-01T20:00:00.000000000' '2010-01-01T21:00:00.000000000'
 '2010-01-01T22:00:00.000000000' '2010-01-01T23:00:00.000000000']
Pre-processing ERA5 data
Pre-processing station data
dim_yc inferred from TaskLoader: (8, 6, 1, 1)
dim_yt inferred from TaskLoader: 1
dim_aux_t inferred from TaskLoader: 5
Setting aux_t_mlp_lay

100%|██████████| 24/24 [00:51<00:00,  2.13s/it]


In [ ]:
# pred['dry_bulb']['mean'].to_netcdf(f'{model_dir}/pred_20100101.nc')

In [13]:
for i in range(24):
    pred['dry_bulb']['mean'].isel(time=i).plot()